In [1]:
import spacy
import os
import pickle
import gc
import tqdm

nlp = spacy.load( 'en_core_web_lg' )
#nlp = spacy.load( 'en_core_web_sm' )

In [34]:
import time
import datetime

def get_time( output=True ):
    
    temp = time.time()
    if output:
        now = datetime.datetime.now()
        print( now.strftime( "%Y.%m.%d %H:%M" ) )
        
    return temp

start_time = get_time()

def print_time( start_time, end_time, interval="seconds" ):
    
    if interval == "hours":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 / 60 ) ) )
    if interval == "minutes":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 ) ) )
    else:
        print ( "Time to process: [%s] seconds" % ( str( end_time - start_time ) ) )
        
print_time( start_time, get_time() )

2018.07.05 12:38
2018.07.05 12:38
Time to process: [0.00014901161193847656] seconds


In [2]:
print( nlp.vocab[u'dog'].similarity( nlp.vocab[u'dachshund']) )
print( nlp.vocab[ u'trump' ].similarity( nlp.vocab[ u'president' ] ) )
print( nlp.vocab[ u'trump' ].similarity( nlp.vocab[ u'man' ] ) )

0.624673
0.183959
0.180288


In [6]:
nlp.vocab.__len__

<method-wrapper '__len__' of spacy.vocab.Vocab object at 0x7f44119340c8>

In [75]:
import numpy as np
from numpy.linalg import norm

cosine = lambda v1, v2: np.dot( v1, v2) / ( norm( v1 ) * norm( v2 ) )

In [96]:


# Let's see if it can figure out this analogy
# King is to man as _____ is to woman?
# three_words = [ "king", "man", "", woman" ] # queen
# three_words = [ "president", "man", "", "woman" ] # vice-president
# three_words = [ "scientist", "man", "", "woman" ] # researcher? cool!
# three_words = [ "penis", "man", "", "woman" ] # vagina!
# three_words = [ "pussy", "woman", "", "man" ] # dick!
# three_words = [ "slut", "woman", "", "man" ] # fucked ...WUH?!?
# three_words = [ "whore", "man", "", "woman" ] # slut!
# three_words = [ "cunt", "woman", "", "man" ] # twat, asshole, dick
# three_words = [ "nigger", "negro", "", "caucasian" ] # caucasians, faggot, closeup... WUH?!?
# three_words = [ "bitch", "woman", "", "man" ] # shit, fuckin, dude, fucker
# three_words = [ "douchebag", "man", "", "woman" ] # asshat, douche, moron, dipshit, shithead?
# three_words = [ "dude", "man", "", "woman" ] # chick, girl, lady, bitch
three_words = [ "chick", "woman", "", "man" ]

word_1st = nlp.vocab[ three_words[ 0 ] ]
word_2nd = nlp.vocab[ three_words[ 1 ] ]
word_4th = nlp.vocab[ three_words[ 3 ] ]

result = word_1st.vector - word_2nd.vector + word_4th.vector
#print( result )

# gather all known words, take only the lowercased versions
allWords = list({w for w in nlp.vocab if w.has_vector and w.is_lower and w.lower_ not in three_words})

# sort by similarity to the result
allWords.sort( key=lambda w: cosine( w.vector, result ) )
allWords.reverse()
print("\n----------------------------\nTop 7 closest results:")
for word in allWords[:7]:   
    print( word.orth_ )


----------------------------
Top 7 closest results:
dude
dudes
chicks
fucking
ass
guy
dick


In [79]:
len( [ w for w in nlp_mine.vocab ] )

24880

In [4]:
def get_similar_tokens( corpus, target_word, threshold ):
    
    #counter = 0
    candidates = []
    for word in keys_list:

        #if counter > 48:
        #    break
        #counter += 1

        score = corpus.vocab[ target_word ].similarity( corpus.vocab[ word ] )
        if score > threshold:
            candidates.append( target_word + "=" + word + ":" + str( score ) )
        
    return candidates


In [24]:
# From: https://github.com/explosion/spaCy/issues/276
def get_foo( word, count=10 ):
    filtered_words = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
    return similarity[:count]

In [101]:
#candidates = get_similar_tokens( nlp, "trump", .20 )
#candidates = get_foo( nlp.vocab[ u'orgasm' ], 25  )
candidates = get_foo( nlp.vocab[ u'microloan' ], 25  )

for word in candidates:
    print( word.lower_ )

loaning
crowdfunding
downpayment
forgivable
lending
borrowers
endowment
repaid
kickstarter
seedbox
tulpa
lika
snowballed
många
borrower
repayment
nonprofit
loans
revolving
farmable
philanthropy
loan
grants
b.s
subprime


In [26]:
embedding_keys_path = "embeddings/trump-tweats-w-links-n-ats-take-III.keys"

with open( embedding_keys_path, 'r' ) as keys_file:
    
    # omit newline char: https://stackoverflow.com/questions/12330522/reading-a-file-without-newlines
    keys_list = keys_file.read().splitlines()

print( "keys_list", len( keys_list) )

keys_list 24498


In [27]:
embedding_path = "embeddings/trump-tweats-w-links-n-ats-take-III.glove"
print( "Loading embeddings %s" % embedding_path )
embedding_matrix = pickle.load( open( embedding_path, "rb" ) )
embedding_matrix.shape

Loading embeddings embeddings/trump-tweats-w-links-n-ats-take-III.glove


(24498, 300)

In [28]:
from spacy.vectors import Vectors

nlp_mine = spacy.blank( 'en' )
nlp_mine.vocab.vectors = Vectors( data=embedding_matrix, keys=keys_list )
print( nlp_mine.vocab.vectors.shape )


(24498, 300)


In [30]:
print( nlp_mine.vocab[ u'trump' ].similarity( nlp_mine.vocab[ u'president' ] ) )
print( nlp_mine.vocab[ u'trump' ].similarity( nlp_mine.vocab[ u'man' ] ) )
print( nlp_mine.vocab[ u'trump' ].similarity( nlp_mine.vocab[ u'tower' ] ) )
print( nlp_mine.vocab[ u'trump' ].similarity( nlp_mine.vocab[ u'condo' ] ) )

0.0845074392332
0.117482802258
0.255267771416
0.359575176797


In [42]:
def get_similar_tokens( which_nlp, target_word, threshold ):
    
    start_time = get_time()
    candidates = []
    
    for word in keys_list:

        score = which_nlp.vocab[ target_word ].similarity( which_nlp.vocab[ word ] )
        if score > threshold:
            candidates.append( ( word, score ) )
    
    # order tuples by score: https://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples
    candidates.sort( key=lambda candidate: candidate[ 1 ], reverse=True ) 
    
    print_time( start_time, get_time() )
    
    return candidates

candidates = get_similar_tokens( nlp_mine, "trump", 0.30 )

for word, score in candidates:
    print( word, score )

2018.07.05 12:50
2018.07.05 12:50
Time to process: [0.45949363708496094] seconds
trump 1.0
Trump 1.0
TRUMP 1.0
Melania 0.45651504795
MELANIA 0.45651504795
Casino 0.452227508974
casino 0.452227508974
Hilton 0.42345148615
Trumps 0.414337546526
TRUMPS 0.414337546526
trumps 0.414337546526
Ivanka 0.406093466256
Resorts 0.399282742864
resorts 0.399282742864
Wynn 0.390242137461
casinos 0.38784101066
Mahal 0.387827881163
Taj 0.385192201156
Donald 0.383422199146
DONALD 0.383422199146
developer 0.372490767499
condo 0.359575176797
Vegas 0.358032071965
VEGAS 0.358032071965
Resort 0.349366960007
resort 0.349366960007
RESORT 0.349366960007
Hotel 0.347573866331
hotel 0.347573866331
HOTEL 0.347573866331
Caesars 0.340374644378
Las 0.322768330185
LAS 0.322768330185
LAs 0.322768330185
Golf 0.321984426767
golf 0.321984426767
hotels 0.321549088326
Hotels 0.321549088326
Celebrity 0.319644958539
celebrity 0.319644958539
CELEBRITY 0.319644958539
condominiums 0.314406215866
Forbes 0.307812816973
card 0.3039540

In [49]:
candidates = get_similar_tokens( nlp_mine, "sexting", 0.25 )

for word, score in candidates:
    print( word, score )

2018.07.05 13:01
2018.07.05 13:01
Time to process: [0.4554331302642822] seconds
sexting 1.0
Sexting 1.0
LAWFARE 0.311345438394
Dobias 0.311253037462
tweeting 0.310225709636
TWEETING 0.310225709636
Teens 0.291672429767
Robocall 0.288734581852
ROBOCALL 0.288734581852
robocall 0.288734581852
Weiner 0.288649663682
Razza 0.283476015574
unpresidential 0.280662965542
isnt 0.279528299606
Isnt 0.279528299606
ISNT 0.279528299606
somethings 0.277073138006
hashtag 0.275318835977
3046 0.27137843937
Lovechild 0.270264826063
Tramonto 0.270144537406
50000 0.269773160923
Irans 0.266205130831
Phn 0.265648541795
Levins 0.265339200918
Løkke 0.264317001079
2403 0.263824651171
bullying 0.262553091185
17000 0.262050410741
Omarosa 0.261874226171
Damelin 0.260122833545
Lumby 0.257776647928
Bho 0.257736243193
BHO 0.257736243193
Vlog 0.25742266697
Rager 0.255976719465
Colene 0.255871179715
Sletten 0.254230518538
surveil 0.252530839795
Trayvon 0.252441366648
Drumstick 0.251036462737
Kreig 0.25102915199
Parl 0.250

In [59]:
candidates = get_similar_tokens( nlp_mine, "luxury", 0.25 )

for word, score in candidates:
    print( word, score )

2018.07.05 13:11
2018.07.05 13:11
Time to process: [0.4380040168762207] seconds
luxury 1.0
Luxury 1.0
luxurious 0.717888515317
Luxurious 0.717888515317
hotels 0.600007280332
Hotels 0.600007280332
Hotel 0.567024141717
hotel 0.567024141717
HOTEL 0.567024141717
cars 0.540415184296
Cars 0.540415184296
suites 0.502500365843
Suites 0.502500365843
condominiums 0.488497434043
priced 0.48536386749
expensive 0.482812929355
Expensive 0.482812929355
villas 0.478832750085
brand 0.475219068391
Brand 0.475219068391
car 0.472071542177
Car 0.472071542177
Condos 0.470885191258
boutique 0.468374915804
Resorts 0.448167389216
resorts 0.448167389216
premium 0.447225841038
PREMIUM 0.447225841038
Premium 0.447225841038
Mercedes 0.440655880125
goods 0.436876808575
Resort 0.428569483884
resort 0.428569483884
RESORT 0.428569483884
jewelry 0.426608762221
Jewelry 0.426608762221
SUVs 0.425439477877
shopping 0.424567213665
Shopping 0.424567213665
opulent 0.421503256466
condo 0.420629538309
dining 0.417735837687
Dini

In [27]:
candidates = get_similar_tokens( "obama", 0.40 )
for word in candidates:
    print( word )

obama=Obama:1.0
obama=Barack:0.92547219582
obama=US:0.416175979137
obama=Hillary:0.649791608994
obama=President:0.445708892591
obama=us:0.416175979137
obama=Clinton:0.70856035988
obama=Romney:0.531305474516
obama=Republicans:0.53889969238
obama=election:0.413482947169
obama=Washington:0.450644435657
obama=campaign:0.493692594791
obama=debate:0.459067934419
obama=Democrats:0.560830458121
obama=House:0.404474491686
obama=speech:0.475875558041
obama=vote:0.409736218505
obama=Republican:0.478441961796
obama=Bush:0.757098791234
obama=Vote:0.409736218505
obama=Senate:0.475979668486
obama=Debate:0.459067934419
obama=White:0.41686724997
obama=Congress:0.420323951513
obama=Gop:0.460374901582
obama=week:0.413215873849
obama=leaders:0.416115479053
obama=GOP:0.460374901582
obama=president:0.445708892591
obama=polls:0.43500934389
obama=saying:0.416835073225
obama=Senator:0.538339592759
obama=plan:0.412532910811
obama=candidate:0.418609397269
obama=Administration:0.530525565123
obama=VOTE:0.40973621

In [30]:
candidates = get_similar_tokens( "Clinton", 0.40 )
for word in candidates:
    print( word )

Clinton=that:0.436019995415
Clinton=Obama:0.70856035988
Clinton=has:0.401115519932
Clinton=he:0.42292296606
Clinton=but:0.418666834348
Clinton=should:0.411166340496
Clinton=Barack:0.612398962703
Clinton=now:0.401951585428
Clinton=Hillary:0.692514053879
Clinton=would:0.413885481268
Clinton=what:0.427112079879
Clinton=He:0.42292296606
Clinton=President:0.549198187124
Clinton=last:0.403753168978
Clinton=If:0.403241456909
Clinton=him:0.417927917382
Clinton=Clinton:1.0
Clinton=if:0.403241456909
Clinton=Romney:0.443267250573
Clinton=even:0.458548130487
Clinton=interview:0.408684957549
Clinton=Bill:0.517069500701
Clinton=What:0.427112079879
Clinton=Republicans:0.612489619751
Clinton=election:0.425275015668
Clinton=Washington:0.524292098236
Clinton=campaign:0.533878879935
Clinton=debate:0.485226446482
Clinton=Now:0.401951585428
Clinton=Democrats:0.587225541983
Clinton=House:0.504525878006
Clinton=say:0.420557312752
Clinton=speech:0.511209961967
Clinton=vote:0.428409611158
Clinton=support:0.429

In [31]:
candidates = get_similar_tokens( "Pence", 0.40 )
for word in candidates:
    print( word )

Pence=Pence:1.0
Pence=Rose:0.41822954375
Pence=share:0.417077539644
Pence=fell:0.40407342919
Pence=cents:0.505148085458
Pence=pence:1.0
Pence=rose:0.41822954375
Pence=PENCE:1.0
Pence=SHARE:0.417077539644
Pence=shares:0.483522537554
Pence=Share:0.417077539644


In [32]:
candidates = get_similar_tokens( "Putin", 0.40 )
for word in candidates:
    print( word )

Putin=Obama:0.419637820223
Putin=President:0.473754361392
Putin=Clinton:0.431702679126
Putin=Russia:0.619720125241
Putin=Bush:0.555893119254
Putin=leaders:0.425382055435
Putin=president:0.473754361392
Putin=saying:0.46330910568
Putin=visit:0.42971172742
Putin=Prime:0.486428477051
Putin=Minister:0.410939854436
Putin=Putin:1.0
Putin=Russian:0.601302176084
Putin=Summit:0.435486820738
Putin=talks:0.41399081474
Putin=promised:0.415765893669
Putin=Russians:0.439813324929
Putin=Moscow:0.618918831565
Putin=presidency:0.403113323776
Putin=Presidency:0.403113323776
Putin=Visit:0.42971172742
Putin=Ukraine:0.473330073598
Putin=Assad:0.422141295665
Putin=Leaders:0.425382055435
Putin=CLINTON:0.431702679126
Putin=PRESIDENT:0.473754361392
Putin=Talks:0.41399081474
Putin=BUSH:0.555893119254
Putin=Mubarak:0.44221879389
Putin=RUSSIA:0.619720125241
Putin=Vladimir:0.72623916325
Putin=Merkel:0.518561680206
Putin=prime:0.486428477051
Putin=OBAMA:0.419637820223
Putin=Karzai:0.456150698087
Putin=Chavez:0.42022

In [32]:
def get_related( word ):
    
    filtered_words = [ w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15 ]
    #similarity = sorted( filtered_words, key=lambda w: word.similarity( w ), reverse=True )
    #return similarity[ :10 ]
    return filtered_words

print( [ w.lower_ for w in get_related( nlp.vocab[ u'president' ] ) ] )

[]


In [33]:
nlp.vocab[ u'president' ]

In [44]:
nlp2 = spacy.load( 'en_core_web_lg', parser=False )
print( nlp2.vocab[u'dog'].similarity(nlp2.vocab[u'dachshund']) )
print( nlp2.vocab[ u'trump' ].similarity( nlp2.vocab[ u'president' ] ) )
print( nlp2.vocab[ u'trump' ].similarity( nlp2.vocab[ u'man' ] ) )

0.624673
0.183959
0.180288


In [6]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher( nlp.vocab )
matcher.add( 'OBAMA', None, nlp( u"Barack Obama" ) )
doc = nlp( u"Barack Obama lifts America one last time in emotional farewell" )
matches = matcher(doc)

matches

[(7732777389095836264, 0, 2)]

In [8]:
import spacy

nlp = spacy.load( 'en' )  # make sure to use larger model!
tokens = nlp( u'dog cat banana' )

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))


dog dog 1.0
dog cat 0.53907
dog banana 0.28761
cat dog 0.53907
cat cat 1.0
cat banana 0.487522
banana dog 0.28761
banana cat 0.487522
banana banana 1.0


In [9]:
import spacy

nlp = spacy.load( 'en_core_web_lg' )  # make sure to use larger model!
tokens = nlp( u'dog cat banana' )

for token1 in tokens:
    for token2 in tokens:
        print( token1.text, token2.text, token1.similarity( token2 ) )

dog dog 1.0
dog cat 0.801686
dog banana 0.243276
cat dog 0.801686
cat cat 1.0
cat banana 0.281544
banana dog 0.243276
banana cat 0.281544
banana banana 1.0


In [11]:
tokens = nlp( u'Obama Clinton Criminal Trump' )

for token1 in tokens:
    for token2 in tokens:
        print( token1.text, token2.text, token1.similarity( token2 ) )

Obama Obama 1.0
Obama Clinton 0.782694
Obama Criminal 0.246474
Obama Trump 0.42475
Clinton Obama 0.782694
Clinton Clinton 1.0
Clinton Criminal 0.219609
Clinton Trump 0.415443
Criminal Obama 0.246474
Criminal Clinton 0.219609
Criminal Criminal 1.0
Criminal Trump 0.217229
Trump Obama 0.42475
Trump Clinton 0.415443
Trump Criminal 0.217229
Trump Trump 1.0


In [16]:
doc = nlp( u'Apple is looking at buying U.K. startup for $1 billion' )
#doc = nlp( u"In any event, Washington is broken, and our country is in serious trouble and total disarray.  Very simple.  Politicians are all talk, no action.  They are all talk and no action.  And it's constant; it never ends." )

for token in doc:
    print( token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

Apple apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True False
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True False
buying buy VERB VBG pcomp xxxx True False
U.K. u.k. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True False
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [15]:
doc = nlp( u'Apple is looking at buying U.K. startup for $1 billion' )

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [20]:
# THIS HANGS. PROBABLY NEEDS PORT 5000 tunneled... NOPE!
from spacy import displacy

doc = nlp(u'This is a sentence.')
displacy.serve(doc, style='dep')


    Serving on port 5000...
    Using the 'dep' visualizer


    Shutting down server on port 5000.



In [21]:
doc = nlp( u'Hello, world. Here are two sentences.' )
print([t.text for t in doc])

['Hello', ',', 'world', '.', 'Here', 'are', 'two', 'sentences', '.']


In [25]:
doc = nlp( u"Peach emoji is where it has always been. Peach is the superior emoji. It's outranking eggplant 🍑 ")
print(doc[0].text)          # Peach
print(doc[1].text)          # emoji
print(doc[-1].text)         # 🍑
print(doc[17:19].text)      # outranking eggplant

noun_chunks = list( doc.noun_chunks )
print( "start?" )
for chunk in noun_chunks:
    print( chunk.text )  # Peach emoji
print( "end?" )
sentences = list(doc.sents )
assert len(sentences) == 3
print(sentences[1].text)    # 'Peach is the superior emoji.'

Peach
emoji
🍑
outranking eggplant
start?
Peach emoji
it
Peach
the superior emoji
It
eggplant 🍑
end?
Peach is the superior emoji.


In [26]:
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')
apple = doc[0]
print('Fine-grained POS tag', apple.pos_, apple.pos )
print('Coarse-grained POS tag', apple.tag_, apple.tag )
print('Word shape', apple.shape_, apple.shape )
print('Alphanumeric characters?', apple.is_alpha )
print('Punctuation mark?', apple.is_punct )

billion = doc[10]
print('Digit?', billion.is_digit)
print('Like a number?', billion.like_num)
print('Like an email address?', billion.like_email)

Fine-grained POS tag PROPN 95
Coarse-grained POS tag NNP 15794550382381185553
Word shape Xxxxx 16072095006890171862
Alphanumeric characters? True
Punctuation mark? False
Digit? False
Like a number? True
Like an email address? False


In [27]:
doc = nlp(u'San Francisco considers banning sidewalk delivery robots')
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

from spacy.tokens import Span
doc = nlp(u'FB is hiring a new VP of global policy')
doc.ents = [Span(doc, 0, 1, label=doc.vocab.strings[u'ORG'])]
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

San Francisco 0 13 GPE
FB 0 2 ORG


In [28]:
from spacy import displacy

doc_dep = nlp(u'This is a sentence.')
displacy.serve(doc_dep, style='dep')

doc_ent = nlp(u'When Sebastian Thrun started working on self-driving cars at Google '
              u'in 2007, few people outside of the company took him seriously.')
displacy.serve(doc_ent, style='ent')


    Serving on port 5000...
    Using the 'dep' visualizer


    Shutting down server on port 5000.


    Serving on port 5000...
    Using the 'ent' visualizer


    Shutting down server on port 5000.



In [29]:
doc = nlp(u"Apple and banana are similar. Pasta and hippo aren't.")

apple = doc[0]
banana = doc[2]
pasta = doc[6]
hippo = doc[8]

print('apple <-> banana', apple.similarity(banana))
print('pasta <-> hippo', pasta.similarity(hippo))
print(apple.has_vector, banana.has_vector, pasta.has_vector, hippo.has_vector)

apple <-> banana 0.583184
pasta <-> hippo 0.0793491
True True True True


In [30]:
doc = nlp(u"When Sebastian Thrun started working on self-driving cars at Google "
          u"in 2007, few people outside of the company took him seriously.")

dep_labels = []
for token in doc:
    while token.head != token:
        dep_labels.append(token.dep_)
        token = token.head
print(dep_labels)

['advmod', 'advcl', 'compound', 'nsubj', 'advcl', 'nsubj', 'advcl', 'advcl', 'xcomp', 'advcl', 'prep', 'xcomp', 'advcl', 'npadvmod', 'amod', 'pobj', 'prep', 'xcomp', 'advcl', 'punct', 'amod', 'pobj', 'prep', 'xcomp', 'advcl', 'amod', 'pobj', 'prep', 'xcomp', 'advcl', 'pobj', 'prep', 'xcomp', 'advcl', 'prep', 'xcomp', 'advcl', 'pobj', 'prep', 'xcomp', 'advcl', 'prep', 'xcomp', 'advcl', 'pobj', 'prep', 'xcomp', 'advcl', 'punct', 'amod', 'nsubj', 'nsubj', 'advmod', 'nsubj', 'prep', 'advmod', 'nsubj', 'det', 'pobj', 'prep', 'advmod', 'nsubj', 'pobj', 'prep', 'advmod', 'nsubj', 'dobj', 'advmod', 'punct']


In [35]:
def put_spans_around_tokens(doc):
    """Here, we're building a custom "syntax highlighter" for
    part-of-speech tags and dependencies. We put each token in a
    span element, with the appropriate classes computed. All whitespace is
    preserved, outside of the spans. (Of course, HTML will only display
    multiple whitespace if enabled – but the point is, no information is lost
    and you can calculate what you need, e.g. <br />, <p> etc.)
    """
    output = []
    html = '<span class="{classes}">{word}</span>{space}'
    for token in doc:
        if token.is_space:
            output.append(token.text)
        else:
            classes = 'pos-{} dep-{}'.format(token.pos_, token.dep_)
            output.append(html.format(classes=classes, word=token.text, space=token.whitespace_))
    string = ''.join(output)
    string = string.replace('\n', '')
    string = string.replace('\t', '    ')
    return '<pre>{}</pre>'.format(string)

doc = nlp(u"This is a test.\n\nHello   world.")
html = put_spans_around_tokens(doc)


from IPython.core.display import display, HTML
html
#display( HTML( html ) )

'<pre><span class="pos-DET dep-nsubj">This</span> <span class="pos-VERB dep-ROOT">is</span> <span class="pos-DET dep-det">a</span> <span class="pos-NOUN dep-attr">test</span><span class="pos-PUNCT dep-punct">.</span><span class="pos-INTJ dep-nmod">Hello</span>   <span class="pos-NOUN dep-ROOT">world</span><span class="pos-PUNCT dep-punct">.</span></pre>'

## Look at Tweet Summaries

22322